## NB outdated!

The following examples will work, however the approach is outdated, and the functionality is marked for deprecation

## Create a  PowerOps Client

A power ops client is a layer on top the regular CogniteClient that contains domain specific interface against CDF and the powerops data models. See quickstart guide for setting up. 

In [ ]:
import os

from cognite.client import CogniteClient

from cognite.powerops import PowerOpsClient

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"  # settings files are not in current dir

power = PowerOpsClient.from_settings()
# The regular cognite client is available under the .cdf attribute.
client: CogniteClient = power.cdf

Note it is important that the `CogShop` version have been set to trigger SHOP runs.

## Fetching SHOP Runs

### Listing SHOP runs

In [ ]:
shop_runs = power.shop.list(limit=5, watercourse="Fornebu")

In [ ]:
shop_runs

### Listing with filters 
* Filtering on the start time and end time. The expected format is `YYYY-MM-DD` as seen in this example: 
* The there is also the option to filter on multiple watercourses by passing a list

In [ ]:
power.shop.list(limit=5, start_after="2022-01-01", watercourse=["Fornebu"])

### Retrieving SHOP run(s)

In [ ]:
run = power.shop.retrieve(shop_runs[0].external_id)

In [ ]:
run

### Retrieving latest SHOP run


In [ ]:
power.shop.retrieve_latest(watercourse="Fornebu", latest_by="created_time")

In [ ]:
power.shop.retrieve_latest(watercourse="Fornebu", latest_by="created_time", end_before="2023-09-01")

### Inspecting SHOP Run Input Files

The `SHOPRun` object gives access to the input files and produced outputs (in the case of a successful run).

In [ ]:
from IPython.display import Markdown, display

In [ ]:
# A small helper to illustrate
def display_yaml(yaml_str):
    display(Markdown(f"```yaml\n{yaml_str}```"))

In [ ]:
run.check_status()

In [ ]:
display_yaml(run.get_case_file()[:500])

In [ ]:
for content in run.get_shop_files():
    display_yaml(content[:50])

### Inspecting SHOP Run Output Files

In [ ]:
for log_file, content in run.get_log_files():
    display(Markdown(f"**{log_file}**"))
    display_yaml(content[:500])

## Trigger SHOP Run

Here we are reusing an old case file for demo purposes

In [ ]:
import time

from cognite.powerops.client.shop.data_classes.shop_case import SHOPCase
from cognite.powerops.client.shop.data_classes.shop_run import SHOPRunStatus

In [ ]:
new_case_data = run.get_case_file()
new_case = SHOPCase(new_case_data, watercourse="Fornebu")

In [ ]:
MAX_WAIT_IN_SECONDS = 120

In [ ]:
t0 = time.time()

new_run = power.shop.trigger_single_casefile(new_case)
# Waiting for SHOP run to finish
while (time.time() - t0) < MAX_WAIT_IN_SECONDS:
    if new_run.check_status() is SHOPRunStatus.SUCCESS:
        print(f"RUN is done in {time.time() -t0:.2f} seconds")
        break
    time.sleep(1)
    print(".", end="")

In [ ]:
for log_file, content in new_run.get_log_files():
    display(Markdown(f"**{log_file}**"))
    display_yaml(content[:500])

In [ ]:
new_run

For more information about the SHOP client, see the `client` section in the reference documentation.

End of notebook.